In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
import random
import matplotlib.pyplot as plt

CLASS_NAMES = ['Building', 'Clutter', 'Vegetation', 'Water', 'Background', 'Car']
COLOR_TO_CLASS = {
    (230, 25, 75): 0,
    (145, 30, 180): 1,
    (60, 180, 75): 2,
    (245, 130, 48): 3,
    (255, 255, 255): 4,
    (0, 130, 200): 5,
    (255, 0, 255): 6
}

IGNORE_COLOR = (255, 0, 255)
NUM_CLASSES = 6
IMPORTANT_CLASSES = [0, 1, 3, 5]

CLASS_TO_COLOR = {
    0: (230, 25, 75),
    1: (145, 30, 180),
    2: (60, 180, 75),
    3: (245, 130, 48),
    4: (255, 255, 255),
    5: (0, 130, 200),
}

def compute_keep_prob_refined(row):
    ratios = {cls: row[f"{i}: {cls}"] for i, cls in enumerate(CLASS_NAMES)}

    if ratios['Background'] >= 0.999:
        return 0.0

    presence_flags = np.array([ratios[cls] > 0.01 for cls in CLASS_NAMES if cls != 'Background'], dtype=np.float32)
    num_present = np.sum(presence_flags)
    non_bg_ratios = np.array([ratios[cls] for cls in CLASS_NAMES if cls != 'Background'])
    std_dev = np.std(non_bg_ratios)

    diversity_score = (num_present / 5.0) - (0.5 * std_dev)
    rare_score = (
        3.0 * ratios['Car'] +
        2.0 * ratios['Water'] +
        1.5 * ratios['Building']
    )

    final_score = diversity_score + rare_score
    keep_prob = 1 / (1 + np.exp(-8 * (final_score - 0.15)))
    return keep_prob

def load_image_paths(df, image_dir, elevation_dir, label_dir):
    image_paths = []
    elevation_paths = []
    label_paths = []
    tile_ids = []
    keep_probs = []

    for _, row in df.iterrows():
        tile_id = row['tile_id']
        image_paths.append(os.path.join(image_dir, f"{tile_id}-ortho.png"))
        elevation_paths.append(os.path.join(elevation_dir, f"{tile_id}-elev.npy"))
        label_paths.append(os.path.join(label_dir, f"{tile_id}-label.png"))
        tile_ids.append(tile_id)
        keep_probs.append(np.float32(row['keep_prob']) if 'keep_prob' in row else np.float32(1.0))

    return image_paths, elevation_paths, label_paths, tile_ids, keep_probs

def augment_image(rgb, elev, label):
    if tf.rank(label) == 2:
        label = tf.expand_dims(label, axis=-1)
    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.flip_left_right(rgb)
        elev = tf.image.flip_left_right(elev)
        label = tf.image.flip_left_right(label)
    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.flip_up_down(rgb)
        elev = tf.image.flip_up_down(elev)
        label = tf.image.flip_up_down(label)
    label = tf.squeeze(label, axis=-1)
    return rgb, elev, label

def _load_npy_elev(path):
    return np.expand_dims(np.load(path.decode("utf-8")), axis=-1).astype(np.float32)

def decode_coloured_label(label_rgb):
    label_rgb = tf.cast(label_rgb, tf.uint8)
    label_flat = tf.reshape(label_rgb, [-1, 3])
    colors = tf.constant(list(COLOR_TO_CLASS.keys()), dtype=tf.uint8)
    class_ids = tf.constant(list(COLOR_TO_CLASS.values()), dtype=tf.int32)
    mask = tf.reduce_all(tf.equal(tf.expand_dims(label_flat, 1), colors), axis=2)
    indices = tf.argmax(tf.cast(mask, tf.int32), axis=1)
    mapped = tf.gather(class_ids, indices)
    return tf.reshape(mapped, [tf.shape(label_rgb)[0], tf.shape(label_rgb)[1]])

def build_tf_dataset(df, image_dir, elevation_dir, label_dir,
                     input_type='rgb', batch_size=32, split='train',
                     augment=False, shuffle=True, tile_size=256):

    image_paths, elevation_paths, label_paths, tile_ids, keep_probs = load_image_paths(df, image_dir, elevation_dir, label_dir)

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, elevation_paths, label_paths, tile_ids, keep_probs))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths), reshuffle_each_iteration=(split == 'train'))

    if split == 'train':
        def filter_fn(rgb_path, elev_path, label_path, tile_id, keep_prob):
            return tf.random.uniform([]) < keep_prob
        dataset = dataset.filter(filter_fn)

    def map_fn(rgb_path, elev_path, label_path, tile_id, keep_prob):
        if input_type == 'rgb':
            return parse_tile(rgb_path, label_path, tile_id, split, augment, tile_size)
        elif input_type == 'rgb_elev':
            return parse_elevation(rgb_path, elev_path, label_path, tile_id, split, augment, tile_size)
        else:
            raise ValueError(f"Unsupported input_type: {input_type}")

    dataset = dataset.map(map_fn, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    dataset = dataset.cache()

    return dataset

def parse_tile(rgb_path, label_path, tile_id, split='train', augment=False, tile_size=256):
    rgb = tf.io.read_file(rgb_path)
    rgb = tf.image.decode_png(rgb, channels=3)
    rgb = tf.image.convert_image_dtype(rgb, tf.float32)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=3)
    label = decode_coloured_label(label)

    if split == 'train' and augment:
        rgb, dummy_elev, label = augment_image(rgb, tf.zeros_like(rgb[..., :1]), label)

    label_onehot = tf.one_hot(tf.cast(label, tf.int32), depth=NUM_CLASSES)
    input_image = tf.image.resize(rgb, [tile_size, tile_size])
    label_onehot = tf.image.resize(label_onehot, [tile_size, tile_size], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return input_image, label_onehot

def parse_elevation(rgb_path, elev_path, label_path, tile_id, split='train', augment=False, tile_size=256):
    rgb = tf.io.read_file(rgb_path)
    rgb = tf.image.decode_png(rgb, channels=3)
    rgb = tf.image.convert_image_dtype(rgb, tf.float32)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=3)
    label = decode_coloured_label(label)

    elev = tf.numpy_function(_load_npy_elev, [elev_path], tf.float32)
    elev.set_shape([None, None, 1])

    if split == 'train' and augment:
        rgb, elev, label = augment_image(rgb, elev, label)

    label_onehot = tf.one_hot(label, depth=NUM_CLASSES)
    input_image = tf.concat([rgb, elev], axis=-1)
    input_image = tf.image.resize(input_image, [tile_size, tile_size])
    label_onehot = tf.image.resize(label_onehot, [tile_size, tile_size], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return input_image, label_onehot

def plot_class_distribution_from_dataset(dataset, num_batches=1000, title="Sampled Training Class Distribution"):
    total_counts = np.zeros(NUM_CLASSES, dtype=np.int64)

    for i, (_, y_batch) in enumerate(dataset.take(num_batches)):
        labels = np.argmax(y_batch.numpy(), axis=-1)
        for cls in range(NUM_CLASSES):
            total_counts[cls] += np.sum(labels == cls)

    pixel_props = total_counts / np.sum(total_counts)

    class_labels = [f"{i}: {CLASS_NAMES[i]}" for i in range(NUM_CLASSES)]
    colours = [np.array(CLASS_TO_COLOR[i]) / 255.0 for i in range(NUM_CLASSES)]

    plt.figure(figsize=(10, 5))
    bars = plt.bar(class_labels, pixel_props, color=colours, edgecolor='black')
    plt.title(title)
    plt.xlabel("Class")
    plt.ylabel("Proportion")
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)

    for bar, prop in zip(bars, pixel_props):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f"{prop:.2%}",
                 ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.show()
